# India government ad spends

### Cleaning Source File

In a recent question in LokSabha, Indian government has provided the details of total expenditure incurred by government on advertisements in newspapers, tv, magazines and digital media. The question detail can be found [here](http://164.100.47.194/Loksabha/Questions/QResult15.aspx?qref=20473&lsno=17) and the answer to that question can be found [here](http://www.davp.nic.in/writereaddata/Lok_Sabha_Question_No_2053.pdf?utm_source=pocket_mylist)

The data provided is in the pdf format. In this notebook, I attempt to clean that data and convert them into csv's for easy analysis.

In [19]:
# loading required packages
import requests # for downloading pdf
from tabula import read_pdf # for converting pdfs to table
import pandas as pd
import os

In [6]:
# download pdf from site
url = "http://www.davp.nic.in/writereaddata/Lok_Sabha_Question_No_2053.pdf"

response = requests.get(url=url, stream=True)

with open('Lok_Sabha_Question_No_2053.pdf', 'wb') as file:
    file.write(response.content)

In [7]:
# use tabula to read pdfs into dataframe
tables = read_pdf('Lok_Sabha_Question_No_2053.pdf', pages='all')

In [8]:
# splitting pdf into three separate file for newspaper, TV and digital ad spends

def concat_df(dflist):
    cols = dflist[0].columns 
    dfs = []
    for df in dflist:
        df.columns = cols 
        dfs.append(df)
    return pd.concat(dfs)


# newspaper ad spends pages are from: 1 to 108
# television ad spends pages are from: 109 to 115
# online ad spends pages are from: 116 to 119

newspaper = concat_df(tables[0:107])
tv = concat_df(tables[108:114])
online = concat_df(tables[115:118])

In [9]:
newspaper.head()

,SL NO,NEWSPAPER NAME,LANGUAGE,PERIODICITY,PUBLICATION CITY,COMMITMENT AMOUNT,Unnamed: 0,Unnamed: 1
0,NaN,NaN,NaN,NaN,NaN,2017-2018,2018-2019,2019-2020
1,1.0,ARTHIK LIPI,BENGALI,DAILY(M),PORT BLAIR,"1,211,750","624,989","316,512"
2,2.0,INFO INDIA,HINDI,DAILY(M),PORT BLAIR,"673,676","667,265","66,545"
3,3.0,SANMARG,HINDI,DAILY(M),PORT BLAIR,0,"272,756","165,675"
4,4.0,THE ANDAMAN EXPRESS,ENGLISH,DAILY(M),PORT BLAIR,"1,042,171","485,418","168,674"


In [10]:
online.head()

,S. No.,Agency Name,2017-18,2018-19,2019-20
0,1,91 MOBILES.COM,"606,710","2 ,563,676","2 23,193"
1,2,AAJTAK.IN,-,-,-
2,3,ABPANANDA.IN,-,-,"6 3,026"
3,4,ABPASMITA.IN,-,-,"6 3,026"
4,5,ABPLIVE.IN,"2 ,743,403","4 ,458,588","8 36,375"


In [11]:
tv.head()

,TELEVISION,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,Channel,2017-18,2018-19,2019-20
1,1.0,& PICTURES,21611,8344847,0
2,2.0,& TV,6042,0,0
3,3.0,10 TV,128226,3176139,0
4,4.0,24 Ghanta,9110505,15058131,3732472


The header of dataframes appears to be misplaced, half of the header is in first row. I will try to combine that and remove the first row.

In [16]:
years = ['2017-2018', '2018-2019', '2019-2020']

newspaper.columns = list(newspaper.columns[:-3]) + years
newspaper = newspaper.iloc[1:, :]

tv.columns = ['Television', 'Channel'] + years
tv = tv.iloc[1:, :]

In [17]:
newspaper.head()

,SL NO,NEWSPAPER NAME,LANGUAGE,PERIODICITY,PUBLICATION CITY,2017-2018,2018-2019,2019-2020
1,1.0,ARTHIK LIPI,BENGALI,DAILY(M),PORT BLAIR,"1,211,750","624,989","316,512"
2,2.0,INFO INDIA,HINDI,DAILY(M),PORT BLAIR,"673,676","667,265","66,545"
3,3.0,SANMARG,HINDI,DAILY(M),PORT BLAIR,0,"272,756","165,675"
4,4.0,THE ANDAMAN EXPRESS,ENGLISH,DAILY(M),PORT BLAIR,"1,042,171","485,418","168,674"
5,5.0,THE ECHO OF INDIA,ENGLISH,DAILY(M),PORT BLAIR,"2,660,578","2,016,642","1,040,729"


In [18]:
tv.head()

,Television,Channel,2017-2018,2018-2019,2019-2020
1,1.0,& PICTURES,21611,8344847,0
2,2.0,& TV,6042,0,0
3,3.0,10 TV,128226,3176139,0
4,4.0,24 Ghanta,9110505,15058131,3732472
5,5.0,4TV News,2425212,0,0


In [23]:
# save those files as csv
os.makedirs('cleaned',exist_ok=True)

newspaper.to_csv('./cleaned/newspaper_ad_spends.csv', index=False)
tv.to_csv('./cleaned/tv_ad_spends.csv', index=False)
online.to_csv('./cleaned/online_ad_spends.csv', index=False)